In [1]:
import logging
import os

#import wiki_old as w # old wiki
import wiki as w 
 
#from gensim.models import word2vec # for orignal w2v
from localgensim.gensim2.models import word2vec 




#from gensim.models.fasttext import FastText
#from gensim.models.word2vec import Word2Vec # not in use
#from localgensim.gensim2.models.word2vec import Word2Vec # not in use

from tqdm import tqdm

WIKIXML = '/home/manni/data/wiki/en-28102022.bz2'
#WIKIXML = '/home/manni/data/wiki/enwiki-20211120-pages-articles-multistream1.xml-p1p41242.bz2'

In [2]:
print(w.__file__)
print(word2vec.__file__)

/home/manni/ner-s2s/word_embedding/wiki.py
/home/manni/ner-s2s/word_embedding/localgensim/gensim2/models/word2vec.py


In [3]:
import sys
sys.path.append("../../imports/")
import saver as sv

In [4]:
logging.basicConfig(format='[%(asctime)s] %(message)s', level=logging.INFO)
os.makedirs('data/', exist_ok=True)

# Test corpus

In [ ]:
from gensim.test.utils import datapath
from gensim import utils

class MyCorpus(object):
    """An interator that yields sentences (lists of str)."""

    def __iter__(self):
        corpus_path = datapath('lee_background.cor')
        for line in open(corpus_path):
            # assume there's one document per line, tokens separated by whitespace
            yield utils.simple_preprocess(line)

In [ ]:
sentences = MyCorpus()

In [ ]:
logging.info('Training model %s', 'word2vec')
model = Word2Vec(sentences, window=5, sg=1, hs=0, negative=10, size=300, sample=0, 
                 workers=1, iter=1, min_count=1)

logging.info('Training done.')

In [ ]:
sentences = w.WikiSentences(WIKIXML, 'en')

# Conll corpus

In [ ]:
sentences = sv.load("conll_corpus")

In [ ]:
model = word2vec.Word2Vec(sentences, window=5, sg=1, hs=0, negative=5, size=300, sample=0, workers=1, iter=1, min_count=1)

In [ ]:
#emb_file = '/mnt/nfs/resdata0/manni/wiki/conll_spx2g.txt'
emb_file = '/mnt/nfs/resdata0/manni/wiki/conll_w2v.txt'

In [ ]:
logging.info('Save trained word vectors')
with open(emb_file, 'w', encoding='utf-8') as f:
    f.write('%d %d\n' % (len(model.wv.vocab), 300))
    for word in tqdm(model.wv.vocab):
        f.write('%s %s\n' % (word, ' '.join([str(v) for v in model.wv[word]])))
logging.info('Done')

# skip if sentence made

In [5]:
# loc = 'num'|'lr'|'ent'
# pos = True|False
# download latest wiki dump
#w.download_wiki_dump('en', WIKIXML)

# parse wiki dump
wiki_sentences = w.WikiSentences(WIKIXML, 'en',lower=True) # Orignal
#wiki_sentences = w.WikiSentences(WIKIXML, 'en',tokenizer_func='EM',lower=True,pos=False,loc=False)
#wiki_sentences = w.WikiSentences(WIKIXML, 'en',tokenizer_func='DEP',lower=True,pos=False,loc=False)
#wiki_sentences = w.WikiSentences(WIKIXML, 'en',tokenizer_func='UNS',lower=True,pos=False,loc=False)
#wiki_sentences = w.WikiSentences(WIKIXML, 'en',tokenizer_func='UNSEM',lower=True,pos=False,loc=False)

[2022-09-29 11:37:52,761] Parsing wiki corpus Altered...
[2022-09-29 11:38:00,198] adding document #0 to Dictionary(0 unique tokens: [])
[2022-09-29 11:39:15,232] adding document #10000 to Dictionary(558662 unique tokens: ['ability', 'ability#1§', 'able#0', 'abolish', 'abolition§']...)
[2022-09-29 11:40:20,604] adding document #20000 to Dictionary(776301 unique tokens: ['ability', 'ability#1§', 'able#0', 'abolish', 'abolition§']...)
[2022-09-29 11:41:18,449] adding document #30000 to Dictionary(938856 unique tokens: ['ability', 'ability#1§', 'able#0', 'abolish', 'abolition§']...)
[2022-09-29 11:42:09,855] adding document #40000 to Dictionary(1075117 unique tokens: ['ability', 'ability#1§', 'able#0', 'abolish', 'abolition§']...)
[2022-09-29 11:44:33,415] adding document #90000 to Dictionary(1314523 unique tokens: ['ability', 'ability#1§', 'able#0', 'abolish', 'abolition§']...)
[2022-09-29 11:45:21,496] adding document #100000 to Dictionary(1428812 unique tokens: ['ability', 'ability#1§'

[2022-09-29 11:57:01,129] discarding 59394 tokens: [('penumalli', 1), ('pinrayi', 1), ('polba', 1), ('radhakrishnamurthy', 1), ('satgachhia', 1), ('sorbhog', 1), ('spectaculated', 1), ('surthi', 1), ('tammineni', 1), ('theog', 1)]...
[2022-09-29 11:57:01,131] keeping 2000000 tokens which were in no less than 0 and no more than 270000 (=100.0%) documents
[2022-09-29 11:57:04,410] resulting dictionary: Dictionary(2000000 unique tokens: ['ability', 'ability#1§', 'able#0', 'abolish', 'abolition§']...)
[2022-09-29 11:57:04,457] adding document #270000 to Dictionary(2000000 unique tokens: ['ability', 'ability#1§', 'able#0', 'abolish', 'abolition§']...)
[2022-09-29 11:57:37,581] discarding 56204 tokens: [('立命館', 1), ('経営学研究科', 1), ('経営学部', 1), ('経営管理研究科', 1), ('経済学研究科', 1), ('経済学部', 1), ('言語教育情報研究科', 1), ('groutwork', 1), ('ungrouted', 1), ('washup#0', 1)]...
[2022-09-29 11:57:37,583] keeping 2000000 tokens which were in no less than 0 and no more than 280000 (=100.0%) documents
[2022-09-29 1

[2022-09-29 12:04:27,966] adding document #390000 to Dictionary(2000000 unique tokens: ['ability', 'ability#1§', 'able#0', 'abolish', 'abolition§']...)
[2022-09-29 12:04:59,383] discarding 52992 tokens: [('rihmer', 1), ('saxonorum', 1), ('slashchyov', 1), ('carpintis', 1), ('greifvogelvorführung', 1), ('ppark', 1), ('copwen', 1), ('eaglesworth', 1), ('facejacker', 1), ('abhisares', 1)]...
[2022-09-29 12:04:59,385] keeping 2000000 tokens which were in no less than 0 and no more than 400000 (=100.0%) documents
[2022-09-29 12:05:02,516] resulting dictionary: Dictionary(2000000 unique tokens: ['ability', 'ability#1§', 'able#0', 'abolish', 'abolition§']...)
[2022-09-29 12:05:02,565] adding document #400000 to Dictionary(2000000 unique tokens: ['ability', 'ability#1§', 'able#0', 'abolish', 'abolition§']...)
[2022-09-29 12:05:32,928] discarding 52379 tokens: [('perverseraph', 1), ('cueller', 1), ('gaide', 1), ('calymmatobacterium', 1), ('dentocariosa', 1), ('extroquens', 1), ('faceium', 1), (

[2022-09-29 12:11:51,412] resulting dictionary: Dictionary(2000000 unique tokens: ['ability', 'ability#1§', 'able#0', 'abolish', 'abolition§']...)
[2022-09-29 12:11:51,463] adding document #520000 to Dictionary(2000000 unique tokens: ['ability', 'ability#1§', 'able#0', 'abolish', 'abolition§']...)
[2022-09-29 12:12:21,285] discarding 50284 tokens: [('hochzeitsstück', 1), ('waldmärchen', 1), ('fixaren', 1), ('grundeler', 1), ('phillipsons', 1), ('tabbouli', 1), ('cyberbombs', 1), ('dellah', 1), ('theatrology', 1), ('timeonic', 1)]...
[2022-09-29 12:12:21,293] keeping 2000000 tokens which were in no less than 0 and no more than 530000 (=100.0%) documents
[2022-09-29 12:12:25,917] resulting dictionary: Dictionary(2000000 unique tokens: ['ability', 'ability#1§', 'able#0', 'abolish', 'abolition§']...)
[2022-09-29 12:12:25,991] adding document #530000 to Dictionary(2000000 unique tokens: ['ability', 'ability#1§', 'able#0', 'abolish', 'abolition§']...)
[2022-09-29 12:12:56,362] discarding 447

[2022-09-29 12:18:46,020] keeping 2000000 tokens which were in no less than 0 and no more than 650000 (=100.0%) documents
[2022-09-29 12:18:50,657] resulting dictionary: Dictionary(2000000 unique tokens: ['ability', 'ability#1§', 'able#0', 'abolish', 'abolition§']...)
[2022-09-29 12:18:50,737] adding document #650000 to Dictionary(2000000 unique tokens: ['ability', 'ability#1§', 'able#0', 'abolish', 'abolition§']...)
[2022-09-29 12:19:16,779] discarding 39507 tokens: [('berthelseni', 1), ('biarritziana', 1), ('bramensis', 1), ('branscombensis', 1), ('bueltenensis', 1), ('calveti', 1), ('camillae', 1), ('caminosa', 1), ('canalifera', 1), ('capillimargo', 1)]...
[2022-09-29 12:19:16,781] keeping 2000000 tokens which were in no less than 0 and no more than 660000 (=100.0%) documents
[2022-09-29 12:19:20,083] resulting dictionary: Dictionary(2000000 unique tokens: ['ability', 'ability#1§', 'able#0', 'abolish', 'abolition§']...)
[2022-09-29 12:19:20,140] adding document #660000 to Dictionar

...

[2022-09-29 15:23:10,802] keeping 2000000 tokens which were in no less than 0 and no more than 4320000 (=100.0%) documents
[2022-09-29 15:23:15,945] resulting dictionary: Dictionary(2000000 unique tokens: ['ability', 'ability#1§', 'able#0', 'abolish', 'abolition§']...)
[2022-09-29 15:23:16,048] adding document #4320000 to Dictionary(2000000 unique tokens: ['ability', 'ability#1§', 'able#0', 'abolish', 'abolition§']...)
[2022-09-29 15:23:42,864] discarding 35689 tokens: [('latecompressus', 2), ('mahafali', 2), ('mamibillae', 2), ('maniensis', 2), ('nepocranus', 2), ('ominosus', 2), ('palatus', 2), ('pardous', 2), ('perlongus', 2), ('permarginatus', 2)]...
[2022-09-29 15:23:42,866] keeping 2000000 tokens which were in no less than 0 and no more than 4330000 (=100.0%) documents
[2022-09-29 15:23:48,162] resulting dictionary: Dictionary(2000000 unique tokens: ['ability', 'ability#1§', 'able#0', 'abolish', 'abolition§']...)
[2022-09-29 15:23:48,265] adding document #4330000 to Dictionary(20

[2022-09-29 15:30:36,496] discarding 34162 tokens: [('kennealey', 2), ('akhiya', 2), ('palsin', 2), ('tuikong', 2), ('aerilata', 2), ('burgenrunde', 2), ('lils', 2), ('woiski', 2), ('boosenburg', 2), ('arcticisoli', 2)]...
[2022-09-29 15:30:36,498] keeping 2000000 tokens which were in no less than 0 and no more than 4450000 (=100.0%) documents
[2022-09-29 15:30:42,021] resulting dictionary: Dictionary(2000000 unique tokens: ['ability', 'ability#1§', 'able#0', 'abolish', 'abolition§']...)
[2022-09-29 15:30:42,125] adding document #4450000 to Dictionary(2000000 unique tokens: ['ability', 'ability#1§', 'able#0', 'abolish', 'abolition§']...)
[2022-09-29 15:31:15,954] discarding 31050 tokens: [('penright', 2), ('ulatoski', 2), ('åblad', 2), ('kornharber', 2), ('leckström', 2), ('särneman', 2), ('ibéyise', 2), ('chteniia', 2), ('shisong', 2), ('belaia', 2)]...
[2022-09-29 15:31:15,955] keeping 2000000 tokens which were in no less than 0 and no more than 4460000 (=100.0%) documents
[2022-09-2

[2022-09-29 15:37:16,064] keeping 2000000 tokens which were in no less than 0 and no more than 4570000 (=100.0%) documents
[2022-09-29 15:37:21,260] resulting dictionary: Dictionary(2000000 unique tokens: ['ability', 'ability#1§', 'able#0', 'abolish', 'abolition§']...)
[2022-09-29 15:37:21,365] adding document #4570000 to Dictionary(2000000 unique tokens: ['ability', 'ability#1§', 'able#0', 'abolish', 'abolition§']...)
[2022-09-29 15:37:49,669] discarding 37358 tokens: [('macropyge', 2), ('maladensis', 2), ('marginauctum', 2), ('menoparia', 2), ('metabowmania', 2), ('mohawkensis', 2), ('morozoviella', 2), ('neoeridotrypella', 2), ('oryctocare', 2), ('pachycranium', 2)]...
[2022-09-29 15:37:49,671] keeping 2000000 tokens which were in no less than 0 and no more than 4580000 (=100.0%) documents
[2022-09-29 15:37:53,562] resulting dictionary: Dictionary(2000000 unique tokens: ['ability', 'ability#1§', 'able#0', 'abolish', 'abolition§']...)
[2022-09-29 15:37:53,634] adding document #458000

[2022-09-29 15:44:30,849] discarding 34436 tokens: [('azhaikkudhe', 2), ('kuzhandhaiyadi', 2), ('thennai', 2), ('halfhour', 2), ('soberscove', 2), ('contempuls', 2), ('vakilpour', 2), ('kokangol', 2), ('haitatsunin', 2), ('cavungi', 2)]...
[2022-09-29 15:44:30,851] keeping 2000000 tokens which were in no less than 0 and no more than 4700000 (=100.0%) documents
[2022-09-29 15:44:36,023] resulting dictionary: Dictionary(2000000 unique tokens: ['ability', 'ability#1§', 'able#0', 'abolish', 'abolition§']...)
[2022-09-29 15:44:36,128] adding document #4700000 to Dictionary(2000000 unique tokens: ['ability', 'ability#1§', 'able#0', 'abolish', 'abolition§']...)
[2022-09-29 15:45:01,635] discarding 36887 tokens: [('aankda', 2), ('dhulghat', 2), ('skålbrekka', 2), ('veanova', 2), ('lanchowfu', 2), ('蘭州府', 2), ('gravenwiesbach', 2), ('bayankhutag', 2), ('aichfeldes', 2), ('bayabani', 2)]...
[2022-09-29 15:45:01,637] keeping 2000000 tokens which were in no less than 0 and no more than 4710000 (=1

[2022-09-29 15:51:09,758] resulting dictionary: Dictionary(2000000 unique tokens: ['ability', 'ability#1§', 'able#0', 'abolish', 'abolition§']...)
[2022-09-29 15:51:09,830] adding document #4820000 to Dictionary(2000000 unique tokens: ['ability', 'ability#1§', 'able#0', 'abolish', 'abolition§']...)
[2022-09-29 15:51:38,715] discarding 34328 tokens: [('mudickal', 2), ('thandekkad', 2), ('parkkari', 2), ('ezhippuram', 2), ('maramkulangara', 2), ('morakkala', 2), ('nadamel', 2), ('itzamná', 2), ('kafati', 2), ('afrocyclops', 2)]...
[2022-09-29 15:51:38,717] keeping 2000000 tokens which were in no less than 0 and no more than 4830000 (=100.0%) documents
[2022-09-29 15:51:44,208] resulting dictionary: Dictionary(2000000 unique tokens: ['ability', 'ability#1§', 'able#0', 'abolish', 'abolition§']...)
[2022-09-29 15:51:44,313] adding document #4830000 to Dictionary(2000000 unique tokens: ['ability', 'ability#1§', 'able#0', 'abolish', 'abolition§']...)
[2022-09-29 15:52:12,348] discarding 34767

In [6]:
#sv.save(wiki_sentences,"wiki_sentences_pos_sample")
#sv.save(wiki_sentences,"wiki_sentences_pos")
#sv.save(wiki_sentences,"wiki_sentences_dep")
#sv.save(wiki_sentences,"wiki_sentences_sp")
#sv.save(wiki_sentences,"wiki_sentences_sp_loc")
#sv.save(wiki_sentences,"wiki_sentences_sp_ent")
#sv.save(wiki_sentences,"wiki_sentences_sp_ent_sample")
#sv.save(wiki_sentences,"wiki_sentences") # orignal
#sv.save(wiki_sentences,"wiki_sentences_dep2")
#sv.save(wiki_sentences,"wiki_sentences_uns")
#sv.save(wiki_sentences,"wiki_sentences_unsem")
#sv.save(wiki_sentences,"wiki_sentences_em")
#sv.save(wiki_sentences,"wiki_sentences_wnet")
sv.save(wiki_sentences,"wiki_sentences_wnet_noun")

Saved the data


# Phrase mining

In [ ]:
from gensim.test.utils import datapath
from gensim.models.phrases import Phrases

In [ ]:
phrases = Phrases(sentences, min_count=100, threshold=1)
frozen_phrases = phrases.freeze()

In [ ]:
sv.save(phrases,"gensim_phrases")

# Train procedure

In [ ]:
#sentences = sv.load("wiki_sentences_no")
#temp_sens are cased!!
#sentences = sv.load("temp_sens")
 
#sentences = sv.load("wiki_sentences") #Normal sentences using wiki_old.py

#Wiki_Sentences_SP are cased
#sentences = sv.load("Wiki_Sentences_SP")

#sentences = sv.load("wiki_sentences_sp_loc") #New
#sentences = sv.load("wiki_sentences_sp") #New

#sentences = sv.load("wiki_sentences_pos") # not to be used
#sentences = sv.load("Wiki_sentences_pos_sample")

#sentences = sv.load("wiki_sentences_sp_ent") # New
#sentences = sv.load("wiki_sentences_sp_ent_sample") # New

#sentences = sv.load("wiki_sentences_dep") #New
#sentences = sv.load("wiki_sentences_dep2") #New

#wiki english sample Cased 
#sentences = sv.load("Wiki_sentences_sp_sample")
#sentences = sv.load("wiki_sentences_uns") #New
#sentences = sv.load("wiki_sentences_unsem") #New
#sentences = sv.load("wiki_sentences_em") #New
#sentences = sv.load("wiki_sentences_wnet") #New
sentences = sv.load("wiki_sentences_wnet_noun") #New

In [ ]:
for sent in sentences:
    print(sent[:100])
    break

In [ ]:
#sentences = wiki_sentences
print("Minimum length of token:",sentences.wiki.token_min_len)

In [ ]:
logging.info('Training model %s', 'wordnet')
model = word2vec.Word2Vec(sentences, window=10, sg=1, hs=0, negative=5, size=300, workers=40, iter=5)
logging.info('Training done.')

In [ ]:
#emb_file = '/mnt/nfs/resdata0/manni/wiki/en_wiki_SPX2_mc1_epoch5_300_filtered_sample.txt'
#emb_file = '/mnt/nfs/resdata0/manni/wiki/en_wiki_w2v_mc1_epoch5_300_sample.txt'
#emb_file = '/mnt/nfs/resdata0/manni/wiki/en_wiki_SPX2R_mc1_epoch5_300_filtered.txt'
#emb_file = '/mnt/nfs/resdata0/manni/wiki/en_wiki_spx2w2v_mc1_epoch5_300.txt'
#emb_file = '/mnt/nfs/resdata0/manni/wiki/en_wiki_SPX2_mc1_epoch5_300_con1.txt'
#emb_file = '/mnt/nfs/resdata0/manni/wiki/en_wiki_SPX2_mc1_epoch5_300_reversed.txt'
#emb_file = '/mnt/nfs/resdata0/manni/wiki/en_wiki_SPX2_mc100_epoch5_300_reversed.txt'
#emb_file = '/mnt/nfs/resdata0/manni/wiki/en_wiki_SPX2_mc100_epoch5_300_neg10.txt'
#emb_file = '/mnt/nfs/resdata0/manni/wiki/en_wiki_SPX2_mc100_epoch5_300_neg10_w3.txt'
#emb_file = '/mnt/nfs/resdata0/manni/wiki/en_wiki_SPX2S_mc100_epoch5_300.txt'
#emb_file = '/mnt/nfs/resdata0/manni/wiki/en_wiki_SPX2B_mc100_epoch5_300_sub3.txt'
#emb_file = '/mnt/nfs/resdata0/manni/wiki/en_wiki_spx2TB_mc100_epoch5_300_LR.txt'
#emb_file = '/mnt/nfs/resdata0/manni/wiki/en_wiki_spx2POS_mc100_epoch5_300.txt'
#emb_file = '/mnt/nfs/resdata0/manni/wiki/en_wiki_spx2DEP_mc100_epoch5_300.txt'
#emb_file = '/mnt/nfs/resdata0/manni/wiki/en_wiki_spx2LRM3_mc100_epoch5_300.txt'
#emb_file = '/mnt/nfs/resdata0/manni/wiki/en_wiki_spx2LOC_mc100_epoch5_300.txt'
#emb_file = '/mnt/nfs/resdata0/manni/wiki/en_wiki_spx2_mc100_epoch5_300.txt'
#emb_file = '/mnt/nfs/resdata0/manni/wiki/en_wiki_spx2_mc100_epoch5_300_w1.txt'
#emb_file = '/mnt/nfs/resdata0/manni/wiki/en_wiki_spx2_mc100_epoch5_300_reversed.txt'
#emb_file = '/mnt/nfs/resdata0/manni/wiki/en_wiki_spx_mc100_epoch5_300_loc.txt'
#emb_file = '/mnt/nfs/resdata0/manni/wiki/en_wiki_w2v_mc100_epoch5_300_w1.txt'
#emb_file = '/mnt/nfs/resdata0/manni/wiki/en_wiki_spx2_mc100_epoch5_300_pos.txt'
#emb_file = '/mnt/nfs/resdata0/manni/wiki/en_wiki_spx_mc100_epoch5_300_ent_w10.txt'
#emb_file = '/mnt/nfs/resdata0/manni/wiki/en_wiki_spx2_mc100_epoch5_300_dep2_w1.txt'
#emb_file = '/mnt/nfs/resdata0/manni/wiki/en_wiki_spx2_mc100_epoch5_300_ent_static_w3.txt'
#emb_file = '/mnt/nfs/resdata0/manni/wiki/en_wiki_spx2_mc100_epoch5_300_uns.txt'
#emb_file = '/mnt/nfs/resdata0/manni/wiki/en_wiki_spx2_mc100_epoch5_300_unsem.txt'
#emb_file = '/home/manni/embs/en_wiki_spx_mc100_epoch5_300_em.txt'
#emb_file = '/mnt/nfs/resdata0/manni/wiki/en_wiki_spx2_mc100_epoch5_300_uns_w1.txt'
#emb_file = '/home/manni/embs/en_wiki_wnet_epoch5_300_w10.txt'
emb_file = '/home/manni/embs/en_wiki_wnet_epoch5_300_w10_exclusive_noun.txt'

In [ ]:
vocab = model.wv.vocab #dict

In [ ]:
len(vocab)

In [ ]:
vocab.pop('[', None)
vocab.pop(']', None)
len(vocab)


In [ ]:
_vocab = list(vocab.keys())
vocab = set()
for word in _vocab:
    if '§' in word:
        continue
    vocab.add(word)
len(vocab)

In [ ]:
logging.info('Save trained word vectors')
with open(emb_file, 'w', encoding='utf-8') as f:
    f.write('%d %d\n' % (len(vocab), 300))
    for word in tqdm(vocab, position=0):
        f.write('%s %s\n' % (word, ' '.join([str(v) for v in model.wv[word]])))
logging.info('Done')